# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [4]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(dotenv_path="ironhack/.env")

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY') 

In [5]:
#%pip install langchain-huggingface


### Example 1

#### Create our QandA application

In [6]:

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import LLMChain


In [7]:
from langchain_community.document_loaders import CSVLoader
import os

file = os.path.join(os.getcwd(), 'data', 'OutdoorClothingCatalog_1000.csv')
loader = CSVLoader(
    file_path=file,
    csv_args={"delimiter": ","},
    encoding='utf-8-sig'  # <-- aggiunto qui
)
data = loader.load()


In [8]:
# !pip install --upgrade --force-reinstall sentence-transformers

In [9]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})
).from_loaders([loader])

C:\Users\rospi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

#### Coming up with test datapoints

In [11]:
data[10]

Document(metadata={'source': 'c:\\Users\\rospi\\Desktop\\Ironhack\\lab-langchain-evaluation\\data\\OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\r\n\r\nSize & Fit\r\n- Pants are Favorite Fit: Sits lower on the waist.\r\n- Relaxed Fit: Our most generous fit sits farthest from the body.\r\n\r\nFabric & Care\r\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\r\n\r\nAdditional Features\r\n- Relaxed fit top with raglan sleeves and rounded hem.\r\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\r\n\r\nImported.")

In [12]:
data[11]

Document(metadata={'source': 'c:\\Users\\rospi\\Desktop\\Ironhack\\lab-langchain-evaluation\\data\\OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

#### Hard-coded examples

In [13]:
from langchain.prompts import PromptTemplate

In [14]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Examples:\n"
             "1. Query: Do the Cozy Comfort Pullover Set have side pockets?\n"
             "   Answer: Yes\n"
             "2. Query: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?\n"
             "   Answer: The DownTek collection\n"
             "Query: {query}\n"
             "Answer:"
)

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        # Split the response to get the answer
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
# llm = OpenAI()
llm = ChatOpenAI()

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Example query
query = "Is the Cozy Comfort Pullover Set available in different colors?"

# Run the chain
result = llm_chain.run({"query": query})

# Print the result
print(result)


C:\Users\rospi\AppData\Local\Temp\ipykernel_22520\1856079041.py:46: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
C:\Users\rospi\AppData\Local\Temp\ipykernel_22520\1856079041.py:56: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run({"query": query})


answer='Yes, it is available in five different colors: black, grey, navy, pink, and purple.'


#### LLM-Generated examples

In [15]:
from langchain.evaluation.qa import QAGenerateChain

In [16]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [17]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [18]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

C:\Users\rospi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\chains\llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [19]:
new_examples[0]

{'qa_pairs': {'query': "What are some key features of the Women's Campside Oxfords as described in the document?",
  'answer': "The Women's Campside Oxfords are made of soft canvas material for a broken-in feel and look. They feature a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, a moderate arch contour, an EVA foam midsole for cushioning and support, and a chain-tread-inspired molded rubber outsole with a modified chain-tread pattern. The approximate weight of the shoes is 1 lb. 1 oz. per pair."}}

In [20]:
data[0]

Document(metadata={'source': 'c:\\Users\\rospi\\Desktop\\Ironhack\\lab-langchain-evaluation\\data\\OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.")

In [21]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': "What are some key features of the Women's Campside Oxfords as described in the document?",
  'answer': "The Women's Campside Oxfords are made of soft canvas material for a broken-in feel and look. They feature a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, a moderate arch contour, an EVA foam midsole for cushioning and support, and a chain-tread-inspired molded rubber outsole with a modified chain-tread pattern. The approximate weight of the shoes is 1 lb. 1 oz. per pair."},
 {'query': 'Where is the Recycled Waterhog dog mat made?',
  'answer': 'The Recycled Waterhog dog mat is made in the USA.'},
 {'query': "What features does the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece offer?",
  'answer': "The toddler's two-piece swimsuit features bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protect

#### Combine examples

In [22]:
# examples += new_example
examples += d_flattened

In [23]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'answer': 'Yes'}

In [24]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Based on the provided context, the Cozy Comfort Pullover Set does not have side pockets. It has side seam pockets, a back zip pocket, two elastic mesh water bottle pockets, and a top compartment with a pocket with a double-seal zipper for quick access.'}

### Manual Evaluation - Fun part

In [25]:
import langchain
langchain.debug = True

In [26]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": "Side seam pockets and back zip pocket, with mesh insert for quick drainage.<<<<>>>>>Two elastic mesh water bottle pockets.\r\nTop compartment includes pocket with double-seal zipper for quick access.\r\nSide<<<<>>>>>All pockets have sturdy pocket bags and offer plenty of room for a wallet, cell phone and more.\r\n\r\nGusseted crotch for ease of movement.\r\n\r\nImported.<<<<>>>>>Two elastic mesh water bottle pockets.\r\nTop compartment includes pocket with double-se"
}
[llm/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain > llm

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Based on the provided context, the Cozy Comfort Pullover Set does not have side pockets. It has side seam pockets, a back zip pocket, two elastic mesh water bottle pockets, and a top compartment with a pocket with a double-seal zipper for quick access.'}

In [27]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [28]:
examples += d_flattened

In [29]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What are some key features of the Women's Campside Oxfords as described in the document?",
  'answer': "The Women's Campside Oxfords are made of soft canvas material for a broken-in feel and look. They feature a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, a moderate arch contour, an EVA foam midsole for cushioning and support, and a chain-tread-inspired molded rubber outsole with a modified chain-tread pattern. The approximate weight of the shoes is 1 lb. 1 oz. per pair."},
 {'query': 'Where is the Recycled Waterhog dog mat made?',
  'answer': 'The Recycled Waterhog dog mat is made in the USA.'},
 {'query': "What features does the Infant and Toddler Girls' Coastal 

In [30]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [31]:
predictions

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes',
  'result': 'Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'},
 {'query': "What are some key features of the Women's Campside Oxfords as described in the document?",
  'answer': "The Women's Campside Oxfords are made of soft canvas material for a broken-in feel and look. They feature a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, a moderate arch contour, an EVA foam midsole for cushioning and support, and a chain-tread-inspired molded rubber outsole with a modified chain-tread pattern. The approximate wei

In [32]:
from langchain.evaluation.qa import QAEvalChain

In [33]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [34]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [35]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [36]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.

Example 2:
Question: What are some key features of the Women's Campside Oxfords as described in the document?
Real Answer: The Women's Campside Oxfords are made of soft canvas material for a broken-in feel and look. They feature a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, a moderate arch contour, an EVA foam midsole for cushioning and support, and a chain-tread-inspired molded rubber outsole with a modified chain-tread patte

### Example 2
One can also easily evaluate your QA chains with the metrics offered in ragas

In [18]:

from langchain_community.document_loaders import CSVLoader
import os

file = os.path.join(os.getcwd(), 'data', 'nyc_text.txt')
loader = CSVLoader(
    file_path=file,
    csv_args={"delimiter": ","},
    encoding='utf-8-sig'  # <-- aggiunto qui
)
data = loader.load()

In [19]:
from langchain_huggingface import HuggingFaceEmbeddings
loader = CSVLoader(
    file_path=file,
    csv_args={"delimiter": ","},
    encoding='utf-8-sig'  # <-- aggiunto qui
)
index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})).from_loaders([loader])


llm = ChatOpenAI(temperature= 0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

C:\Users\rospi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\indexes\vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [20]:
# testing it out

question = "How did New York City get its name?"
result = qa_chain.invoke({"query": question})
result["result"]

'New York City was named in honor of the Duke of York, who would become King James II of England. It was named in 1664 when England seized the city from Dutch control.'

In [21]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was named in honor of the Duke of York, who would become King James II of England. It was named in 1664 when England seized the city from Dutch control.',
 'source_documents': [Document(id='bed1c94f-b78c-4a00-a165-bbeba705dc60', metadata={'source': 'c:\\Users\\rospi\\Desktop\\Ironhack\\lab-langchain-evaluation\\data\\nyc_text.txt', 'row': 0}, page_content="804: the Duke of York. The city was regained by the Dutch in July 1673 and was renamed New Orange for one year and three months; the city has been continuously named New York since November 1674. New York City was the capital of the United States from 1785 until 1790\n190 distributed over 300.46 square miles (778.2 km2): and has been the largest U.S. city since 1790. The Statue of Liberty greeted millions of immigrants as they came to the U.S. by ship in the late 19th and early 20th centuries\nNew York City is the most densely populated major city in the Unite

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [23]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [49]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain). 
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [24]:
result = qa_chain.invoke({"query": eval_questions[1]})
result["result"]

'The borough of Brooklyn has the highest population in New York City.'

In [38]:
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [39]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'The borough of Brooklyn has the highest population in New York City.',
 'contexts': [Document(id='2244b72c-f7c9-4c2e-a4aa-83ac5a8f7463', metadata={'source': 'c:\\Users\\rospi\\Desktop\\Ironhack\\lab-langchain-evaluation\\data\\nyc_text.txt', 'row': 33}, page_content="New York: Manhattan Island is loosely divided into the Lower\noften called New York City or NYC: Midtown\nis the most populous city in the United States. With a 2020 population of 8: and Uptown regions. Uptown Manhattan is divided by Central Park into the Upper East Side and the Upper West Side\n804: and above the park is Harlem\n190 distributed over 300.46 square miles (778.2 km2): bordering the Bronx (Bronx County).\nNew York City is the most densely populated major city in the United States and more than twice as populous as Los Angeles: None\nthe nation's second-largest city. New York City is located at the southern tip of New York S

In [40]:
#!pip install --no-cache-dir recordclass

In [41]:
#!pip install ragas==0.1.9

In [42]:
from ragas.integrations.langchain import EvaluatorChain 
# from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

# create evaluation chains
faithfulness_chain   = EvaluatorChain(metric=faithfulness)
answer_rel_chain     = EvaluatorChain(metric=answer_relevancy)
context_rel_chain    = EvaluatorChain(metric=context_relevancy)
context_recall_chain = EvaluatorChain(metric=context_recall)

1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [43]:
# Recheck the result that we are going to validate.
result

{'query': 'Which borough of New York City has the highest population?',
 'result': 'The borough of Brooklyn has the highest population in New York City.',
 'source_documents': [Document(id='2244b72c-f7c9-4c2e-a4aa-83ac5a8f7463', metadata={'source': 'c:\\Users\\rospi\\Desktop\\Ironhack\\lab-langchain-evaluation\\data\\nyc_text.txt', 'row': 33}, page_content="New York: Manhattan Island is loosely divided into the Lower\noften called New York City or NYC: Midtown\nis the most populous city in the United States. With a 2020 population of 8: and Uptown regions. Uptown Manhattan is divided by Central Park into the Upper East Side and the Upper West Side\n804: and above the park is Harlem\n190 distributed over 300.46 square miles (778.2 km2): bordering the Bronx (Bronx County).\nNew York City is the most densely populated major city in the United States and more than twice as populous as Los Angeles: None\nthe nation's second-largest city. New York City is located at the southern tip of New Y

**Faithfulness**

In [44]:
%pip install nest_asyncio

import nest_asyncio
nest_asyncio.apply()

# Convert Document objects in 'contexts' to strings
result_fixed = result_updated.copy()
result_fixed["contexts"] = [doc.page_content for doc in result_fixed["contexts"]]

eval_result = faithfulness_chain(result_fixed)
eval_result["faithfulness"]


Note: you may need to restart the kernel to use updated packages.


0.0

In [45]:
# Prepare the input for faithfulness_chain with correct keys and format
fake_result = result.copy()
fake_result["result"] = "we are the champions"

# Map keys and convert source_documents to strings
faithfulness_input = {
	"question": fake_result["query"],
	"answer": fake_result["result"],
	"contexts": [doc.page_content for doc in fake_result["source_documents"]],
}

eval_result = faithfulness_chain(faithfulness_input)
eval_result["faithfulness"]

No statements were generated from the answer.


nan

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [46]:
# Prepare the input for faithfulness_chain with correct keys and format
fake_result = result.copy()
fake_result["result"] = "we are the champions"

# Map keys and convert source_documents to strings
faithfulness_input = {
	"question": fake_result["query"],
	"answer": fake_result["result"],
	"contexts": [doc.page_content for doc in fake_result["source_documents"]],
}

eval_result = faithfulness_chain(faithfulness_input)
eval_result["faithfulness"]

No statements were generated from the answer.


nan

**Context Relevancy**

In [47]:
# Prepare the input for context_recall_chain with correct keys and format
context_recall_input = {
	"question": result["query"],
	"answer": result["result"],
	"contexts": [doc.page_content for doc in result["source_documents"]],
	"ground_truth": ""  # Provide the ground truth answer if available, else leave as empty string
}

eval_result = context_recall_chain(context_recall_input)
eval_result["context_recall"]

0.2

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [48]:
# Prepare the input with correct keys and format for context_recall_chain
fake_result = {
	"question": result["query"],
	"answer": result["result"],
	"contexts": ["I love christmas"],
	"ground_truth": ""  # or provide the actual ground truth if available
}
eval_result = context_recall_chain(fake_result)
eval_result["context_recall"]

0.0

2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [49]:
# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

# evaluate each example/prediction pair using the faithfulness_chain
print("evaluating...")

def prepare_faithfulness_input(example, prediction):
	return {
		"question": example["query"],
		"answer": prediction["result"],
		"contexts": [doc.page_content for doc in prediction["source_documents"]],
	}

results = [faithfulness_chain(prepare_faithfulness_input(ex, pred)) for ex, pred in zip(examples, predictions)]
results

evaluating...


[{'question': 'What is the population of New York City as of 2020?',
  'answer': 'The population of New York City as of 2020 was 8,804,190 residents.',
  'contexts': ["New York: In 2020\noften called New York City or NYC: the city had an estimated population density of 29\nis the most populous city in the United States. With a 2020 population of 8: 302.37 inhabitants per square mile (11\n804: 313.71/km2)\n190 distributed over 300.46 square miles (778.2 km2): rendering it the nation's most densely populated of all larger municipalities (those with more than 100\nNew York City is the most densely populated major city in the United States and more than twice as populous as Los Angeles: 000 residents)\nthe nation's second-largest city. New York City is located at the southern tip of New York State. It constitutes the geographical and demographic center of both the Northeast megalopolis and the New York metropolitan area: with several small cities (of fewer than 100",
   'New York: New York

In [ ]:
# evaluate context recall
print("evaluating...")
r = [context_recall_chain({
	"question": ex["query"],
	"answer": pred["result"],
	"contexts": [doc.page_content for doc in pred["source_documents"]],
	"ground_truth": ex["ground_truths"][0] if "ground_truths" in ex and ex["ground_truths"] else ""
}) for ex, pred in zip(examples, predictions)]
r

evaluating...
